In [1]:
from IPython.display import display
import ipywidgets as widgets
from simpleac import SimPleAC
from monte_carlo import print_monte_carlo_results

button = widgets.Button(description="Run Simulation")
output = widgets.Output()
style = {'description_width': 'initial', 'slider_width': 'initial'}
wing_weight_margin = widgets.FloatSlider(description = "Wing Weight Margin",
                                  value = 1.125,
                                  style = style)
tsfc_margin = widgets.FloatSlider(description = "TSFC Margin", 
                                value = 1,
                                style = style)
takeoff_speed_margin = widgets.FloatSlider(description = "Takeoff Speed Margin",
                                            value = 1,
                                            style = style)
range_margin = widgets.FloatSlider(description = "Range Margin",
                                   value = 1,
                                   style = style)

In [ ]:
def on_button_clicked(b):
    m = SimPleAC()
    m.substitutions.update({
        "W_W_coeff1": wing_weight_margin.value,
        "W_W_coeff2": tsfc_margin.value,
        "m_vmin": takeoff_speed_margin.value,
        "m_range": range_margin.value
    })
    print_monte_carlo_results(m)

In [ ]:
button.on_click(on_button_clicked)
display(wing_weight_margin)
display(tsfc_margin)
display(takeoff_speed_margin)
display(range_margin)
display(button, output)